In [1]:
import pandas as pd
import cleaningtools as ct 
import file_config as fconfig
import merge_config as config
import assay_sample_config as aconfig
import spectral_config as sconfig
import xrf_config as xconfig
import numpy as np
import math
from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()
from tqdm import tqdm
import os
import file_config as fconfig
import pandas as pd

check for pandas
pandas is properly installed
check for tqdm
tqdm is properly installed
check for numpy
numpy is properly installed
check for openpyxl
openpyxl is properly installed


In [3]:
path='/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Lab/XRF/_Daily Master/_XLSX/'
base_path=ct.get_base_path(path,start_point='_AZ_Kay')

base_path
master_path=base_path+'/_Master Databases/xrf samples master.csv'
hole_path=base_path+'/_Lab/XRF/_XRF Hole Masters/'

In [4]:
path='/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Lab/XRF/_Daily Master/_XLSX/'
data_list=[]
for file in os.listdir(path):
    print(file)
    try:
        data=pd.read_excel(path+file)
        data['FileName']=file
        data_list.append(data)
    except Exception as e:
        print(e)

06-12-2022_XRF_KM-22-77.xlsx
06-11-2022_XRF_KM-22-77.xlsx
06-22-2022_XRF_KM-22-80_77 (1).xlsx
06-10-2022_XRF_KM-22-77.xlsx
06-23-2022_XRF_KM-22-80_77 (1).xlsx
6-24-2022_XRF_KM-22-80_77_78.xlsx
06-29-2022_XRF_KM-22-80_77.xlsx
6-25-2022_XRF_KM-22-78_77_80 (1).xlsx
06-26-2022_XRF_KM-22-80_77_penman.xlsx
07-04-2022_XRF_KM-22-82.xlsx
06-28-2022_XRF_KM-22-80_77.xlsx
06-27-2022_XRF_KM-22-80_77.xlsx
07-03-2022_XRF_KM-22-77.xlsx
07-11-2022_XRF_KM-21-25A.xlsx
06-30-2022_XRF_KM-22-80_77.xlsx
07-07-2022_XRF_KM-22-82_27.xlsx
07-05-2022_XRF_KM-22-82.xlsx
07-02-2022_XRF_KM-22-77.xlsx
07-1-2022_XRF_KM-22-80_77.xlsx
07-06-2022_XRF_KM-22-82_42.xlsx
07-08-2022_XRF_KM-21-27.xlsx
6-25-2022_XRF_KM-22-78_77_80.xlsx
06-18-2022_XRF_KM-22-76.xlsx
06-17-2022_XRF_KM-22-76_77.xlsx
06-15-2022_XRF_KM-22-73_77.xlsx
06-14-2022_XRF_KM-22-73.xlsx
06-20-2022_XRF_KM-22-77.xlsx
06-22-2022_XRF_KM-22-80_77.xlsx
06-19-2022_XRF_KM-22-76_77.xlsx
06-13-2022_XRF_KM-22-73.xlsx
06-16-2022_XRF_KM-22-73_77.xlsx
06-21-2022_XRF_KM-22-7

In [5]:
collar_path='/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Drilling/collar master.csv'
collars=pd.read_csv(collar_path)

In [6]:
collars.Hole.unique()

array(['EGH-1', 'EGH-2', 'EGH-3', 'K-1', 'K-10', 'K-10A', 'K-11', 'K-12',
       'K-13', 'K-14', 'K-15', 'K-16', 'K-2', 'K-3', 'K-4', 'K-5', 'K-6',
       'K-7', 'K-8', 'K-9', 'KM-20-01', 'KM-20-02', 'KM-20-03',
       'KM-20-03A', 'KM-20-04', 'KM-20-05', 'KM-20-06', 'KM-20-07',
       'KM-20-09', 'KM-20-10', 'KM-20-10A', 'KM-20-10B', 'KM-20-10C',
       'KM-20-11', 'KM-20-12', 'KM-20-13', 'KM-20-14', 'KM-20-14A',
       'KM-20-15', 'KM-20-16', 'KM-21-17', 'KM-21-18', 'KM-21-18A',
       'KM-21-19', 'KM-21-20', 'KM-21-21', 'KM-21-21A', 'KM-21-22',
       'KM-21-22A', 'KM-21-23', 'KM-21-24', 'KM-21-25', 'KM-21-25A',
       'KM-21-25B', 'KM-21-26', 'KM-21-27', 'KM-21-27A', 'KM-21-27B',
       'KM-21-28', 'KM-21-29', 'KM-21-30', 'KM-21-31', 'KM-21-32',
       'KM-21-33', 'KM-21-34', 'KM-21-35', 'KM-21-36', 'KM-21-37',
       'KM-21-38', 'KM-21-39', 'KM-21-40', 'KM-21-41', 'KM-21-42',
       'KM-21-42A', 'KM-21-42B', 'KM-21-42C', 'KM-21-43', 'KM-21-44',
       'KM-21-45', 'KM-21-46', 'KM-2

In [7]:
xrf=pd.concat(data_list)
xrf=xrf.reset_index(drop=True)
xrf[xrf=='<LOD']=np.nan
xrf[xrf=='na']=np.nan
for map in xconfig.mappings:
    xrf=ct.column_cleanup(xrf,mapping=map)
xrf=ct.drop_no_data(xrf)

##create hole_id column
for format in xconfig.hole_id_formats:
    xrf[format]=xrf.file.str.extract(format,expand=False)
xrf.loc[xrf[xconfig.hole_id_formats[1]].isna(),'hole_id']=xrf.loc[xrf[xconfig.hole_id_formats[1]].isna(),xconfig.hole_id_formats[0]]
xrf.loc[xrf.hole_id.isna(),'hole_id']=xrf.loc[xrf.hole_id.isna(),xconfig.hole_id_formats[1]]

for format in xconfig.hole_id_formats:
    xrf=xrf.drop(format,axis=1)
drop_index=(xrf[xrf['sample_id_xrf'].str.contains('penman')==True]).index
xrf=xrf.drop(drop_index,axis=0)
drop_index=xrf[xrf.hole_id.isna()].index
xrf=xrf.drop(drop_index,axis=0)
## fix hole_ids
fix_hole=xrf['sample_id_xrf'].fillna('').str.split('-').str[0].str.isdigit()
xrf.loc[fix_hole,'hole_id']=(xrf[fix_hole].hole_id.str.split('-').str[:-1].str.join('-')+'-'+xrf[fix_hole].sample_id_xrf.str.split('-').str[0]).values
ends=xrf.sample_id_xrf.str.split('-').str[0].unique()
for end in ends:
    
    hole=collars[collars.Hole.str.split('-').str[-1]==end].Hole.values
    if len(hole)>0:
        print(hole)
        xrf.loc[xrf.sample_id_xrf.str.split('-').str[0]==end,'hole_id']=hole[0]
columns= list(xrf.filter (like='Real').columns)
for i in range(len(columns[:-1])):
    
    drop_index=xrf[xrf[columns[i]]!=xrf[columns[i+1]]].index
    print (f'drop rows {drop_index}')
    xrf=xrf.drop(drop_index,axis=0)

holes=xrf.hole_id.unique()
xrf=xrf.sort_values(['sample_id_xrf','depth_ft','hole_id'])
for hole in holes:
    xrf.loc[xrf.hole_id==hole,'depth_ft']=pd.to_numeric(xrf.loc[xrf.hole_id==hole,'depth_ft'].values,errors='coerce')
    # drop_index=xrf[xrf.hole_id==hole].depth_ft.duplicated().index
    # print(drop_index)
    # xrf=xrf.drop(drop_index,axis=0)
    xrf.loc[xrf.hole_id==hole,'from_ft']=xrf.loc[xrf.hole_id==hole,'depth_ft'].fillna(method='ffill')
    xrf.loc[xrf.hole_id==hole,'to_ft']=xrf.loc[xrf.hole_id==hole,'from_ft'].shift(-1).values-.5
    xrf.loc[xrf.hole_id==hole,'from_m']=xrf.loc[xrf.hole_id==hole,'from_ft'].values*0.3048
    xrf.loc[xrf.hole_id==hole,'to_m']=xrf.loc[xrf.hole_id==hole,'from_m'].shift(-1).values-.1
    
xrf.loc[xrf.from_ft> xrf.to_ft,'to_ft']=xrf.loc[xrf.from_ft> xrf.to_ft,'from_ft']+10
xrf.loc[xrf.from_m> xrf.to_m,'from_m']=xrf.loc[xrf.from_m> xrf.to_m,'from_ft']*0.3048
xrf.loc[xrf.from_m> xrf.to_m,'to_m']=xrf.loc[xrf.from_m> xrf.to_m,'to_ft']*0.3048
xrf=xrf.drop_duplicates(['hole_id','from_ft'],keep='first')



In [10]:
drop=xrf.loc[xrf.from_ft+10< xrf.to_ft].index
xrf=xrf.drop(drop,axis=0)
drop=xrf.loc[xrf.from_m>= xrf.to_m].index
xrf=xrf.drop(drop,axis=0)
print('drop',drop)

Int64Index([ 635,  845, 1581, 1395, 1260, 1278, 1285, 1354, 1591, 1593, 1615,
            1679, 1686, 1140, 1157, 1173, 1190, 1216, 1033, 1035, 1052, 1061,
            1083, 1108,  229,   43,   47,   98,   99,  100,  101,  102,  104,
             106,  145,   13,  810],
           dtype='int64')


In [18]:
xrf.loc[xrf.from_m>= xrf.to_m].index

Int64Index([], dtype='int64')

In [11]:
starting_cols=['sample_id_xrf','depth_ft','from_ft','to_ft','from_m','to_m','hole_id','geo']
ending_cols=[col for col in xrf.columns if col not in starting_cols]

In [12]:
xrf=pd.concat([xrf[starting_cols],xrf[ending_cols]],axis=1)
xrf

,sample_id_xrf,depth_ft,from_ft,to_ft,from_m,to_m,hole_id,geo,Instrument Serial Num,Reading #,...,U User Factor Offset,LE Concentration,LE Error1s,LE User Factor Slope,LE User Factor Offset,QAQC,Real Time 1,Real Time 2,Real Time 3,file
1400,1404.0,NaN,NaN,NaN,NaN,NaN,KM-22-73,Emm,843173.0,22.0,...,0.0,764798.0,474.0,1.0,0.0,CDN-ME-1404,45.0,45.0,45.0,06-14-2022_XRF_KM-22-73.xlsx
1129,1404.0,NaN,NaN,NaN,NaN,NaN,KM-22-76,Emm,843173.0,3.0,...,0.0,725464.0,963.0,1.0,0.0,CDN-ME-1404,45.0,45.0,45.0,06-17-2022_XRF_KM-22-76_77.xlsx
160,1404.0,NaN,NaN,NaN,NaN,NaN,KM-22-77,Joh,843173.0,66.0,...,0.0,742673.0,393.0,1.0,0.0,CDN-ME-1404,45.0,45.0,45.0,06-11-2022_XRF_KM-22-77.xlsx
1986,21-001,1250.0,1250.0,1259.5,381.000,383.948,KM-21-21,Aaro,843173.0,4.0,...,0.0,505379.0,682.0,1.0,0.0,NaN,45.0,45.0,45.0,07-21-2022_KM-21-21A.xlsx
1987,21-002,1260.0,1260.0,1269.5,384.048,386.996,KM-21-21,Aaro,843173.0,5.0,...,0.0,534722.0,704.0,1.0,0.0,NaN,45.0,45.0,45.0,07-21-2022_KM-21-21A.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,82-077,1490.0,1490.0,1499.5,454.152,457.100,KM-22-82,Aaro,843173.0,6.0,...,0.0,499186.0,668.0,1.0,0.0,NaN,45.0,45.0,45.0,07-07-2022_XRF_KM-22-82_27.xlsx
693,82-078,1500.0,1500.0,1510.0,457.200,460.248,KM-22-82,Aaro,843173.0,7.0,...,0.0,497954.0,646.0,1.0,0.0,NaN,45.0,45.0,45.0,07-07-2022_XRF_KM-22-82_27.xlsx
2050,end,NaN,NaN,NaN,NaN,NaN,KM-21-21A,Aaro,843173.0,69.0,...,0.0,878406.0,771.0,1.0,0.0,CDN-BL-10,45.0,45.0,45.0,07-21-2022_KM-21-21A.xlsx
654,end,NaN,NaN,NaN,NaN,NaN,KM-21-25A,Aaro,843173.0,49.0,...,0.0,883459.0,993.0,1.0,0.0,CDN-BL-10,45.0,45.0,45.0,07-11-2022_XRF_KM-21-25A.xlsx


In [13]:
xrf.to_csv(master_path,index=False)

In [14]:
holes=xrf.hole_id.unique()
for hole in holes:

    print(f'create {hole} .xlsx file')
    
    data=xrf[xrf.hole_id==hole]
    
    data.to_excel(hole_path+hole+'_test.xlsx')


create KM-22-73 .xlsx file
create KM-22-76 .xlsx file
create KM-22-77 .xlsx file
create KM-21-21 .xlsx file
create KM-21-25 .xlsx file
create KM-21-27 .xlsx file
create KM-21-28 .xlsx file
create KM-21-42 .xlsx file
create KM-22-78 .xlsx file
create KM-22-80 .xlsx file
create KM-22-81 .xlsx file
create KM-22-82 .xlsx file
create KM-21-21A .xlsx file
create KM-21-25A .xlsx file
create KM-22-28 .xlsx file


In [15]:
xrf=pd.read_csv(master_path)
xrf[xrf.from_m>=xrf.to_m]

,sample_id_xrf,depth_ft,from_ft,to_ft,from_m,to_m,hole_id,geo,Instrument Serial Num,Reading #,...,U User Factor Offset,LE Concentration,LE Error1s,LE User Factor Slope,LE User Factor Offset,QAQC,Real Time 1,Real Time 2,Real Time 3,file
